# Introduction to Quantitative Finance and Financial Risk
## Assignment 1: Triangular and One-way Arbitrage

## Ioannis Michalopoulos
## p3352314

# Quotation Matrix Overview

The quotation matrix represents exchange rates between various currencies, where each row corresponds to the currency being exchanged **from**, and each column corresponds to the currency being exchanged **to**.

Each cell value indicates how many units of the **column currency** one unit of the **row currency** can buy.



In [5]:
import pandas as pd

Quotation_Matrix_df = pd.read_excel('Quotation Matrix.xlsx', index_col=0).T
Quotation_Matrix_df

,AUD,CAD,CHF,GBP,JPY,EUR,USD
USD,1.2440,1.069300,1.135200,0.569900,107.860000,0.713300,NaN
EUR,1.7441,1.499200,1.591500,0.799000,151.220000,NaN,1.399975
JPY,0.0115,0.009900,0.010500,0.005300,NaN,0.006604,0.009267
GBP,2.1829,1.876300,1.992000,NaN,188.396650,1.250564,1.752416
CHF,1.0959,0.942000,NaN,0.501707,95.133448,0.627648,0.880550
CAD,1.1634,NaN,1.060511,0.532325,100.919274,0.666356,0.934630
AUD,NaN,0.858948,0.911216,0.457969,86.939134,0.573247,0.802735


# Triangular Arbitrage Analysis

In this section we implement a Python-based solution for identifying **triangular arbitrage opportunities** using a customized **Quotation Matrix** of currency exchange rates.

### Key Features

- **Matrix Loading**: Automatically reads and formats exchange rate data from an Excel file.
- **Triangular Arbitrage Detection**:
  - Evaluates all possible 3-currency trade cycles.
  - Identifies profitable arbitrage loops based on exchange rate products.
  - Calculates profit both in the original cycle currency and in a user-specified **base currency**.
- **User Interaction**: Prompts the user to choose the base currency to express the final arbitrage profit.




In [1]:
import pandas as pd
from itertools import permutations

def load_matrix(file_path):
    # Load matrix from Excel; assumes first row and first column are currency codes
    df = pd.read_excel(file_path, index_col=0).T
    return df

def triangular_arbitrage(matrix, base_currency):
    currencies = list(matrix.index)
    opportunities = []

    # Generate all permutations of 3 distinct currencies for triangular arbitrage
    for cycle in permutations(currencies, 3):
        start, mid, end = cycle

        # Calculate product of rates: start->mid, mid->end, end->start
        rate1 = matrix.loc[start, mid]
        rate2 = matrix.loc[mid, end]
        rate3 = matrix.loc[end, start]

        if pd.isna(rate1) or pd.isna(rate2) or pd.isna(rate3):
            continue  # skip if any rate is missing

        result = rate1 * rate2 * rate3

        # Only consider arbitrage if result > 1 (profit)
        if result > 1.0001:  # add small threshold to avoid floating error noise
            # Calculate profit starting with 1 unit of base currency
            if start == base_currency:
                profit_in_base = result - 1
            else:
                # Convert the profit back to base_currency using direct rate if available
                if start == base_currency:
                    profit_in_base = result - 1
                else:
                    try:
                        conv_rate = matrix.loc[start, base_currency]
                        profit_in_base = (result - 1) * conv_rate
                    except KeyError:
                        # If no direct conversion, skip conversion
                        profit_in_base = result - 1

            opportunities.append({
                'Cycle': f"{start} → {mid} → {end} → {start}",
                'Product_of_Rates': result,
                'Profit_in_Starting_Currency': result - 1,
                f'Profit_in_{base_currency}': profit_in_base
            })

    return opportunities

def main():
    file_path = 'Quotation Matrix.xlsx'
    matrix = load_matrix(file_path)

    print("Currencies detected:", list(matrix.columns))

    base_currency = input("Enter the currency code to express profit in (e.g., USD): ").strip().upper()
    if base_currency not in matrix.columns:
        print(f"Currency '{base_currency}' not found in matrix. Please check the input.")
        return

    opportunities = triangular_arbitrage(matrix, base_currency)

    if not opportunities:
        print("No triangular arbitrage opportunities found.")
    else:
        print("\nTriangular Arbitrage Opportunities:")
        for opp in opportunities:
            print(f"Cycle: {opp['Cycle']}")
            print(f"  Product of rates: {opp['Product_of_Rates']:.6f}")
            print(f"  Percentage profit: {opp['Profit_in_Starting_Currency']:.6%}")
            print(f"  Profit in starting currency: {opp['Profit_in_Starting_Currency']:.6f}")
            print(f"  Profit in {base_currency}: {opp[f'Profit_in_{base_currency}']:.6f}")
            print("-" * 40)

if __name__ == "__main__":
    main()


Currencies detected: ['AUD', 'CAD', 'CHF', 'GBP', 'JPY', 'EUR', 'USD']

Triangular Arbitrage Opportunities:
Cycle: USD → JPY → GBP → USD
  Product of rates: 1.001782
  Percentage profit: 0.178243%
  Profit in starting currency: 0.001782
  Profit in USD: 0.001782
----------------------------------------
Cycle: USD → CHF → JPY → USD
  Product of rates: 1.000756
  Percentage profit: 0.075579%
  Profit in starting currency: 0.000756
  Profit in USD: 0.000756
----------------------------------------
Cycle: USD → AUD → JPY → USD
  Product of rates: 1.002209
  Percentage profit: 0.220873%
  Profit in starting currency: 0.002209
  Profit in USD: 0.002209
----------------------------------------
Cycle: EUR → JPY → GBP → EUR
  Product of rates: 1.002285
  Percentage profit: 0.228453%
  Profit in starting currency: 0.002285
  Profit in USD: 0.003198
----------------------------------------
Cycle: EUR → AUD → JPY → EUR
  Product of rates: 1.001413
  Percentage profit: 0.141304%
  Profit in startin

## Triangular Arbitrage Results (Indicative Base Currency: USD)

The results show all detected **profitable triangular arbitrage opportunities**  based on the exchange rates in the loaded quotation matrix. Each entry corresponds to a full currency trading cycle where a trader starts and ends with the same currency, and ends up with a net gain.

### Key Information:
- **Cycle**: The sequence of three currency exchanges that completes the arbitrage loop.
- **Product of Rates**: Multiplicative result of the three exchange rates. Values > 1 indicate a profitable cycle.
- **Percentage Profit**: The gain over 1 unit of the starting currency in percentage terms.
- **Profit in USD**: The estimated profit from completing the cycle, converted into USD (as selected by the user).

Only cycles with **net gain (profit > 0)** above a small threshold are shown.

This output can guide traders or analysts in spotting arbitrage routes in currency markets.


# One-Way Arbitrage Detection

This section of the notebook implements a tool to analyze **one-way arbitrage opportunities** between a selected pair of currencies using the loaded **Quotation Matrix**.

### How It Works

- **User Input**:
  - The user selects a currency to **sell** and a currency to **buy**.
- **Path Exploration**:
  - The program checks for both direct and indirect (2-step) conversion paths between the selected currencies.
  - It calculates the **effective exchange rate** for each path.
- **Arbitrage Opportunity**:
  - If an indirect path offers a better rate than the direct one, a **one-way arbitrage opportunity** is identified.
  - The program highlights the best conversion route and quantifies the percentage gain over the direct trade.


This analysis helps uncover whether routing through intermediate currencies results in a better exchange outcome, a classic one-way arbitrage scenario.


In [6]:
import pandas as pd
from itertools import permutations

def load_matrix(file_path):
    return pd.read_excel(file_path, index_col=0).T

def get_effective_rate(path, matrix):
    rate = 1.0
    for i in range(len(path) - 1):
        from_cur = path[i]
        to_cur = path[i + 1]
        r = matrix.loc[from_cur, to_cur]
        if pd.isna(r):
            return None  # Invalid path
        rate *= r
    return rate

def find_one_way_arbitrage(matrix, sell_currency, buy_currency):
    currencies = list(matrix.columns)
    if sell_currency not in currencies or buy_currency not in currencies:
        print("Invalid currency codes.")
        return

    paths = []

    # Allow up to 3-step paths (i.e., 1 or 2 intermediate currencies)
    for intermediate in currencies:
        if intermediate not in [sell_currency, buy_currency]:
            # 2-hop: sell → intermediate → buy
            path = [sell_currency, intermediate, buy_currency]
            rate = get_effective_rate(path, matrix)
            if rate:
                paths.append((path, rate))

    # Also include direct path
    direct_rate = matrix.loc[sell_currency, buy_currency]
    if not pd.isna(direct_rate):
        paths.append(([sell_currency, buy_currency], direct_rate))

    # Sort paths by effective rate (most favorable last currency amount per 1 unit sold)
    paths.sort(key=lambda x: x[1], reverse=True)

    print(f"\nAll possible paths from {sell_currency} to {buy_currency}:")
    for path, rate in paths:
        print(f"  Path: {' → '.join(path)} | Effective Rate: {rate:.6f}")

    if len(paths) > 1:
        best_path = paths[0]
        direct = next(((p, r) for p, r in paths if p == [sell_currency, buy_currency]), None)
        if direct and best_path[1] > direct[1]: #* 1.0001:
            print("\n Arbitrage Opportunity Detected!")
            print(f"  Best Path: {' → '.join(best_path[0])} yields more than direct route")
            print(f"  Gain: {(best_path[1] / direct[1] - 1) * 100:.4f}%")

def main():
    file_path = 'Quotation Matrix.xlsx'
    matrix = load_matrix(file_path)

    print("Available currencies:", list(matrix.columns))

    sell_currency = input("Enter the currency you want to SELL: ").strip().upper()
    buy_currency = input("Enter the currency you want to BUY: ").strip().upper()

    if sell_currency == buy_currency:
        print("Sell and buy currencies must differ.")
        return

    find_one_way_arbitrage(matrix, sell_currency, buy_currency)

if __name__ == "__main__":
    main()


Available currencies: ['AUD', 'CAD', 'CHF', 'GBP', 'JPY', 'EUR', 'USD']

All possible paths from EUR to USD:
  Path: EUR → CHF → USD | Effective Rate: 1.401395
  Path: EUR → JPY → USD | Effective Rate: 1.401302
  Path: EUR → CAD → USD | Effective Rate: 1.401198
  Path: EUR → GBP → USD | Effective Rate: 1.400180
  Path: EUR → AUD → USD | Effective Rate: 1.400050
  Path: EUR → USD | Effective Rate: 1.399975

 Arbitrage Opportunity Detected!
  Best Path: EUR → CHF → USD yields more than direct route
  Gain: 0.1015%


## Example Results: Selling EUR to Buy USD

- **Available currencies** include major world currencies such as AUD, CAD, CHF, GBP, JPY, EUR, and USD.

- The program evaluates all possible paths from **EUR (sell currency)** to **USD (buy currency)**:
  - Both direct exchange and indirect routes via one intermediate currency are considered.
  - The effective exchange rates for these paths indicate how many USD are obtained per 1 EUR sold.

- The paths and their corresponding effective rates are:
  - EUR → CHF → USD: 1.401395
  - EUR → JPY → USD: 1.401302
  - EUR → CAD → USD: 1.401198
  - EUR → GBP → USD: 1.400180
  - EUR → AUD → USD: 1.400050
  - EUR → USD (direct): 1.399975

- **Arbitrage Opportunity Detected!**
  - The best path is through **CHF** (EUR → CHF → USD), yielding a slightly higher rate than the direct EUR → USD exchange.
  - This corresponds to a gain of approximately **0.1015%** over the direct conversion.

### Interpretation:
Routing the currency trade through Swiss Franc (CHF) provides a marginally better return compared to the direct EUR → USD conversion, highlighting a **one-way arbitrage opportunity** that can be exploited for profit.
